In [1]:
#Set up and Load
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', None)
df=pd.read_csv("C:\\Users\Jones Mbela\\Desktop\\RENNY\\AI AND ML\\Lend Secure\\Data\\world_real_estate_data(147k).csv")
print("Dataframe shape:", df.shape)

<>:7: SyntaxWarning: invalid escape sequence '\J'
<>:7: SyntaxWarning: invalid escape sequence '\J'
C:\Users\Jones Mbela\AppData\Local\Temp\ipykernel_34040\3277366043.py:7: SyntaxWarning: invalid escape sequence '\J'
  df=pd.read_csv("C:\\Users\Jones Mbela\\Desktop\\RENNY\\AI AND ML\\Lend Secure\\Data\\world_real_estate_data(147k).csv")


Dataframe shape: (147536, 14)


In [18]:
#Drop Columns
drop_cols=[
    "url",
    "image",
    "title",
    "apartment_bedrooms",
    "apartment_floor",
    "apartment_living_area"
]
df=df.drop(columns=[col for col in drop_cols if col in df.columns])
print("Dataframe shape after dropping columns:", df.shape)

Dataframe shape after dropping columns: (144941, 8)


In [19]:
#Target cleaning
Target= "price_in_USD"
before= len(df)
df= df[df[Target]>0]
after= len (df)
print(f"Removed {before-after} rows with invlaid price.")

Removed 0 rows with invlaid price.


In [21]:
#Fixing Data Types
Area_cols= ["apartment_total_area"]
for col in Area_cols:
    if col in df.columns:
        df[col]=(
            df[col]
            .astype(str)
            .str.replace(r'[^\d\.]',"", regex=True)
            .replace("",np.nan)
            .astype(float)
        )
df[Area_cols].dtypes      

apartment_total_area    float64
dtype: object

In [22]:
#Replacing invliad numeric entries with NaN
current_year= datetime.now().year
#building_construction_year
if "building_construction_year" in df.columns:
    df.loc[
        (df["building_construction_year"]<1800) | 
        (df["building_construction_year"]>current_year),
        "building_construction_year"
    ]= np.nan
#building_total_floors
if "building_total_floors" in df.columns:
    df.loc[df["building_total_floors"]<=0,"building_total_floors"]= np.nan
#apartment_rooms
if "apartment_rooms" in df.columns:
    df.loc[df["apartment_rooms"]<=0,"apartment_rooms"]= np.nan
#aportment_bathrooms
if "apartment_bathrooms" in df.columns:
    df.loc[df["apartment_bathrooms"]<=0,"apartment_bathrooms"]= np.nan

In [23]:
#Handling Missing Values for Numerical Features
num_median_cols=[
    "apartment_total_area",
    "apartment_rooms",
    "building_total_floors",
    "apartment_bathrooms"
]
for col in num_median_cols:
    if col in df.columns:
        df[col].fillna(df[col].median())



In [24]:
#Grouped mediam for construction year by country
if {"building_construction_year","country"}.issubset(df.columns):
    df["building_construction_year"]= (
        df.groupby("country")["building_construction_year"]
        .transform(lambda s: s.fillna(s.median()))
    
    )

C:\Users\Jones Mbela\AppData\Roaming\Python\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Jones Mbela\AppData\Roaming\Python\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [25]:
#Handling Missing Values for Categorical Features
#country(mode)
if "country" in df.columns:
    df["country"]= df["country"].fillna(df["country"].mode()[0])

#location
if "location" in df.columns:
    df["location"]= df["location"].fillna("Unknown")

In [27]:
#outlier capping for numerical features
cap_cols=[
    "apartment_rooms",
    "apartment_bathrooms",
    "building_total_floors",
    "apartment_total_area"

]
for col in cap_cols:
    if col in df.columns:
        cap=df[col].quantile(0.99)
        df[col]= np.where(df[col]>cap, cap, df[col])

In [28]:
#sanity checks
df.isnull().mean().sort_values(ascending=False).head(10)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
building_construction_year,144417.0,2013.242935,18.129325,1800.0,2005.0,2022.0,2023.0,2025.0
building_total_floors,67584.0,8.445712,7.733535,1.0,2.0,5.0,14.0,38.0
apartment_rooms,73579.0,2.556653,1.084265,1.0,2.0,2.0,3.0,6.0
apartment_bathrooms,55781.0,1.351123,0.614789,1.0,1.0,1.0,2.0,4.0
apartment_total_area,140244.0,131.181127,128.261609,1.0,57.0,88.0,150.0,800.0
price_in_USD,144941.0,412229.053615,842142.589153,61.0,105432.0,190212.0,398930.0,30602832.0


In [29]:
#Saving Cleaned Data
df.to_csv("C:\\Users\\Jones Mbela\\Desktop\\RENNY\\AI AND ML\\Lend Secure\\Data\\cleaned_real_estate_data.csv", index=False)
print("Cleaned data saved successfully.")

Cleaned data saved successfully.


In [1]:
#